I deleted the pandas dataframe info and overwrote the Brooklyn csv with empty rows, stupid!
I have to rebuild it...

In [1]:
from selenium import webdriver
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np
import time
import random
# from retrying import retry
import tenacity

In [2]:
listing_type = []
lat = []
lng = []
address = []
no_of_beds = []
no_of_baths = []
sq_area = []
n_hood = []
price = []

url = 'http://streeteasy.com/for-sale/brooklyn/status:listed?refined_search=true'

In [3]:
def page_scrape(page):
    # count = 0
    # On slow connections...
    # result = WebDriverWait(page, 30).until(EC.presence_of_element_located((By.ID, 'result-details')))
    # listings = result.find_element_by_tag_name('ul').find_element_by_tag_name('li')
    listings = page.find_element_by_id('result-details').find_element_by_tag_name('ul').find_elements_by_tag_name('li')
    # collect data here by iterating through each listing and appending to our lists
    for l in listings[:14]:
        # I need an IF statement to test whether the listing is legit before scraping to reduce the amount of N/A values
        # Initiating a counter to help identify at what listing the code breaks, if it does...
        # This has become redundant with the introduction of tenacity retry function
        # count +=1
        
        # longitude and latitude
        g = None
        try:
            g = l.get_attribute('se:map:point')
            if g:
                lt, ln = g.split(',')
                lat.append(float(lt))
                lng.append(float(ln))
            else:
                lat.append('N/A')
                lng.append('N/A')
        except:
            lat.append('N/A')
            lng.append('N/A')
        # time.sleep(1)
        
        # address
        ad = None
        try:
            ad = l.find_element_by_class_name('details-title').text.split('\n')[0]
            if ad:
                address.append(ad)
            else:
                address.append('N/A')
        except:
            address.append('N/A')
        # time.sleep(1)
        
        # price
        p = None
        try:
            p = float(l.find_element_by_class_name('price').text.replace('$','').replace(',', ''))
            if p:
                price.append(p)
            else:
                price.append('N/A')
        except:
            price.append('N/A')
        # time.sleep(1)
        
        # number of beds
        bd_detail = None
        try:
            bd_detail = l.find_element_by_class_name('details_info').find_element_by_tag_name('span')
            if bd_detail.text.find('bed') > 0:
                no_of_beds.append(float(bd_detail.text.split(' ')[0]))
            # do we want this as a string or float? what are the regression/ml requirements?
            else:
                no_of_beds.append('N/A')
        except:
            no_of_beds.append('N/A')
        # time.sleep(1)
        
        # number of baths
        baths = None
        try:
            lstn_details = l.find_element_by_class_name('details_info').find_elements_by_tag_name('span')
            for detail in lstn_details:
                if detail.text.find('bath') > 0:
                    try:
                        baths = float(detail.text.split(' ')[0])
                    except:
                        baths = 'N/A'
        except:
            baths = 'N/A'
        no_of_baths.append(baths)
        # time.sleep(1)
        
        # square area NB: value in previous listing is being appended to next listing. FIX!
        # update: fixed.
        area = None
        try:
            l_details = l.find_element_by_class_name('details_info').find_elements_by_tag_name('span')
            for detail in l_details:
                 if detail.text.find('ft') > 0:
                    area = float(detail.text.split(' ')[0].replace(',', ''))
            if area:
                sq_area.append(area)
            else:
                sq_area.append('N/A')
        except:
            sq_area.append('N/A')
        # time.sleep(1)
        
        # listing type and neighborhood
        l_type = None
        nhood = None
        try:
            area_details = l.find_elements_by_class_name('details_info')[1].text
            l_type, nhood = area_details.split(' in ')
            if l_type:
                listing_type.append(l_type)
            else:
                listing_type.append('N/A')
            if nhood:
                n_hood.append(nhood)
            else:
                n_hood.append('N/A')
        except:
            listing_type.append('N/A')
            n_hood.append('N/A')
        # time.sleep(1)
    # if count == 14:
        # print('Moving on to the next page...')
    # streeteasy introduces a captcha when they suspect scraping. How will this be overridden?
    # fixed by using Firefox in place of Chrome
    
    return listings

In [4]:
def next_page():
    nxt = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'next')))
    # formerly: nxt = listns[-1].find_element_by_class_name('next'); WebDriverWait improves stability
    nxt.click()

In [5]:
@tenacity.retry()
def retry():
    driver.get(driver.current_url)

In [6]:
driver = webdriver.Firefox()
page_crash = 0
driver.get(url)
time.sleep(2) # possible crash here without time.sleep?
listns = page_scrape(driver)
last_page = int(listns[-1].find_elements_by_class_name('page')[-1].text)
# print 'Counter:', counter
next_page()
for i in range(1, last_page):
    try:
        # counter += 1 # increment repetition may occur here if a break occurs in this try loop... avoided by using scraping in except loop
        listns = page_scrape(driver)
        next_page()
    except:
        page_crash += 1
        print 'Retrying...'
        retry()
    finally:
        delay = random.uniform(0.5, 1)
        time.sleep(delay)
driver.close()
dic = {'building type':listing_type, 'latitude':lat, 'longitude':lng, 'address':address, 'beds':no_of_beds, 'baths':no_of_baths, 'area':sq_area, 'neighborhood':n_hood, 'price':price}
data = pd.DataFrame(dic)
data.to_csv('Brooklyn.csv', index=False)

Retrying...
Retrying...


In [11]:
driver.close()

In [55]:
df.head()

,address,area,baths,beds,building type,latitude,longitude,neighborhood,price


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12432 entries, 0 to 12431
Data columns (total 10 columns):
Unnamed: 0       12432 non-null int64
address          12331 non-null object
area             8631 non-null float64
baths            12051 non-null float64
beds             11138 non-null float64
building type    12306 non-null object
latitude         12331 non-null float64
longitude        12331 non-null float64
neighborhood     12306 non-null object
price            12328 non-null float64
dtypes: float64(6), int64(1), object(3)
memory usage: 971.3+ KB


In [5]:
df.describe()

,Unnamed: 0,area,baths,beds,latitude,longitude,price
count,12432.000000,8631.000000,12051.000000,11138.000000,12331.000000,12331.000000,1.232800e+04
mean,6215.500000,1988.776387,2.147374,2.302523,40.687743,-73.839617,3.757602e+06
std,3588.953608,8100.537745,1.391416,1.414179,1.680861,3.549829,5.771997e+06
min,0.000000,1.000000,1.000000,1.000000,0.000000,-77.519600,1.500000e+05
25%,3107.750000,870.500000,1.000000,1.000000,40.735994,-73.994301,9.750000e+05
50%,6215.500000,1354.000000,2.000000,2.000000,40.759899,-73.979698,1.945000e+06
75%,9323.250000,2219.000000,2.500000,3.000000,40.775902,-73.964029,3.999995e+06
max,12431.000000,735000.000000,22.000000,19.000000,40.872398,40.770080,8.500000e+07
